In [1]:
import numpy as np
import os
import pandas as pd

import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import seaborn as sns

import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.layers import SimpleRNN, GRU
from tensorflow.keras.layers import Dropout, InputLayer, Activation
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit

import warnings
warnings.filterwarnings(action='ignore')

#한글설정
import matplotlib.font_manager as fm

font_dirs = ['/usr/share/fonts/truetype/nanum', ]
font_files = fm.findSystemFonts(fontpaths=font_dirs)

for font_file in font_files:
    fm.fontManager.addfont(font_file)
    
# 한글 출력을 위해서 폰트 옵션을 설정합니다.
# "axes.unicode_minus" : 마이너스가 깨질 것을 방지

sns.set(font="NanumBarunGothic",
        rc={"axes.unicode_minus":False},
        style='darkgrid')

#GPU 사용 설정, -1이면 CPU 사용
# os.environ["CUDA_VISIBLE_DEVICES"] = "0, 1"

gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
if gpus:
    try:
        for i in range(len(gpus)):
            tf.config.experimental.set_memory_growth(gpus[i], True)
    except RuntimeError as e:
        # 프로그램 시작시에 메모리 증가가 설정되어야만 합니다
        print(e)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


# 시계열, LSTM

In [3]:
# 2. LSTM model
def lstm_model():
    seed_num = 42 # model_seed
    tf.random.set_seed(seed_num)

    lstm = Sequential()
    lstm.add(InputLayer(input_shape=(x.shape[1],x.shape[2])))
    lstm.add(LSTM(units=128, activation='hard_sigmoid', return_sequences=True))
    lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
    lstm.add(Dropout(0.2))
    lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
    lstm.add(LSTM(units=32, activation='hard_sigmoid', return_sequences=False))
    lstm.add(Dropout(0.2))
    lstm.add(Dense(units=1, activation='sigmoid'))
    return lstm

def sorted_ls(path):
    mtime=lambda f: os.stat(os.path.join(path, f)).st_mtime
    return list(sorted(os.listdir(path), key=mtime))

path = '/project/LSH/'
with tf.device('/device:GPU:0'):
    # 1. Data load
    x = np.load(path + 'x_(7727,10,4068).npy')
    y = np.load(path + 'y_(7727,1).npy')

    # 3-1. Best model saving
    MODEL_SAVE_FOLDER_PATH = './models/'
    if not os.path.exists(MODEL_SAVE_FOLDER_PATH):
        os.mkdir(MODEL_SAVE_FOLDER_PATH)

    data={}
    sss = StratifiedShuffleSplit(n_splits=10, test_size = 0.2, random_state = 42)

    # 4. Crossvalidation
    for seed, (train_index, test_index) in tqdm(enumerate(sss.split(x, y))):
        X_train, y_train = x[train_index,:,:], y[train_index]
        X_test, y_test = x[test_index,:,:], y[test_index]

        # 3-2. Best model saving
        model_path = MODEL_SAVE_FOLDER_PATH + f'/seed={seed}_'+'{epoch:02d}'+'_val_loss:{val_loss:.4f}.hdf5'

        cb_checkpoint = ModelCheckpoint(filepath=model_path, monitor='val_loss',
                                        verbose=1, save_best_only=True)
        early_stop = EarlyStopping(monitor='val_loss', patience=50, verbose=1, restore_best_weights=False)


        model = lstm_model()
        model.compile(optimizer= keras.optimizers.Adam(learning_rate = 0.001), 
                      loss = "binary_crossentropy", metrics=['acc'])
        model.fit(X_train, y_train, validation_split=0.25, batch_size=128, epochs=500,
                  callbacks=[early_stop, cb_checkpoint], shuffle=False)

        # 5. accuracy checking 
#         from pathlib import Path
#         paths = sorted(Path(MODEL_SAVE_FOLDER_PATH).iterdir(), key=os.path.getmtime)[-1]
#         best_model_path = str(paths)
        best_model_path = MODEL_SAVE_FOLDER_PATH + sorted_ls('models/')[-1]
        print('############# model path : ', best_model_path)

        best_model = keras.models.load_model(best_model_path) 
#         y_pred_train = best_model.predict(X_train, batch_size=10000, workers=-1, use_multiprocessing=True)
        y_pred_test = best_model.predict(X_test, batch_size=10000, workers=-1, use_multiprocessing=True)

        y_pred_test[y_pred_test>0.5]=1
        y_pred_test[y_pred_test<=0.5]=0

        acc = accuracy_score(y_test, y_pred_test)
        data[seed]=acc
        print(f'정확도 : {acc}, seed_num = {seed}')

    df = pd.DataFrame.from_dict(data, orient='index')
    print(f'정확도 df 만들고 평균 확인 : {df.mean().values}')    

0it [00:00, ?it/s]

Epoch 1/500
37/37 [==============================] - 6s 79ms/step - loss: 0.6753 - acc: 0.6004 - val_loss: 0.6704 - val_acc: 0.6054

Epoch 00001: val_loss improved from inf to 0.67041, saving model to ./models/seed=0_01_val_loss:0.6704.hdf5
Epoch 2/500
37/37 [==============================] - 2s 66ms/step - loss: 0.6696 - acc: 0.6097 - val_loss: 0.6541 - val_acc: 0.6054

Epoch 00002: val_loss improved from 0.67041 to 0.65411, saving model to ./models/seed=0_02_val_loss:0.6541.hdf5
Epoch 3/500
37/37 [==============================] - 3s 68ms/step - loss: 0.5885 - acc: 0.6783 - val_loss: 0.5441 - val_acc: 0.7296

Epoch 00003: val_loss improved from 0.65411 to 0.54410, saving model to ./models/seed=0_03_val_loss:0.5441.hdf5
Epoch 4/500
37/37 [==============================] - 2s 68ms/step - loss: 0.4601 - acc: 0.7970 - val_loss: 0.5490 - val_acc: 0.7400

Epoch 00004: val_loss did not improve from 0.54410
Epoch 5/500
37/37 [==============================] - 2s 67ms/step - loss: 0.4107 - ac


Epoch 00040: val_loss did not improve from 0.53970
Epoch 41/500
37/37 [==============================] - 3s 68ms/step - loss: 0.0695 - acc: 0.9838 - val_loss: 1.0698 - val_acc: 0.7245

Epoch 00041: val_loss did not improve from 0.53970
Epoch 42/500
37/37 [==============================] - 3s 69ms/step - loss: 0.0646 - acc: 0.9853 - val_loss: 1.0991 - val_acc: 0.7193

Epoch 00042: val_loss did not improve from 0.53970
Epoch 43/500
37/37 [==============================] - 3s 68ms/step - loss: 0.0636 - acc: 0.9864 - val_loss: 1.0996 - val_acc: 0.7173

Epoch 00043: val_loss did not improve from 0.53970
Epoch 44/500
37/37 [==============================] - 2s 67ms/step - loss: 0.0600 - acc: 0.9877 - val_loss: 1.1140 - val_acc: 0.7135

Epoch 00044: val_loss did not improve from 0.53970
Epoch 45/500
37/37 [==============================] - 2s 67ms/step - loss: 0.0550 - acc: 0.9886 - val_loss: 1.1372 - val_acc: 0.7167

Epoch 00045: val_loss did not improve from 0.53970
Epoch 46/500
37/37 [===

37/37 [==============================] - 3s 69ms/step - loss: 0.1269 - acc: 0.9646 - val_loss: 0.8488 - val_acc: 0.7393

Epoch 00021: val_loss did not improve from 0.48256
Epoch 22/500
37/37 [==============================] - 3s 69ms/step - loss: 0.1178 - acc: 0.9672 - val_loss: 0.8817 - val_acc: 0.7445

Epoch 00022: val_loss did not improve from 0.48256
Epoch 23/500
37/37 [==============================] - 3s 68ms/step - loss: 0.1117 - acc: 0.9728 - val_loss: 0.9112 - val_acc: 0.7393

Epoch 00023: val_loss did not improve from 0.48256
Epoch 24/500
37/37 [==============================] - 3s 69ms/step - loss: 0.1039 - acc: 0.9737 - val_loss: 0.9418 - val_acc: 0.7419

Epoch 00024: val_loss did not improve from 0.48256
Epoch 25/500
37/37 [==============================] - 3s 69ms/step - loss: 0.0962 - acc: 0.9769 - val_loss: 0.9715 - val_acc: 0.7419

Epoch 00025: val_loss did not improve from 0.48256
Epoch 26/500
37/37 [==============================] - 3s 68ms/step - loss: 0.1029 - acc:

37/37 [==============================] - 2s 67ms/step - loss: 0.6718 - acc: 0.6011 - val_loss: 0.6474 - val_acc: 0.6171

Epoch 00002: val_loss improved from 0.66844 to 0.64737, saving model to ./models/seed=2_02_val_loss:0.6474.hdf5
Epoch 3/500
37/37 [==============================] - 3s 69ms/step - loss: 0.5979 - acc: 0.6781 - val_loss: 0.5098 - val_acc: 0.7665

Epoch 00003: val_loss improved from 0.64737 to 0.50979, saving model to ./models/seed=2_03_val_loss:0.5098.hdf5
Epoch 4/500
37/37 [==============================] - 3s 70ms/step - loss: 0.4723 - acc: 0.7881 - val_loss: 0.4720 - val_acc: 0.7820

Epoch 00004: val_loss improved from 0.50979 to 0.47196, saving model to ./models/seed=2_04_val_loss:0.4720.hdf5
Epoch 5/500
37/37 [==============================] - 3s 68ms/step - loss: 0.3939 - acc: 0.8378 - val_loss: 0.4883 - val_acc: 0.7833

Epoch 00005: val_loss did not improve from 0.47196
Epoch 6/500
37/37 [==============================] - 3s 68ms/step - loss: 0.3555 - acc: 0.857

...

37/37 [==============================] - 3s 70ms/step - loss: 0.0559 - acc: 0.9868 - val_loss: 1.1701 - val_acc: 0.7225

Epoch 00051: val_loss did not improve from 0.51122
Epoch 52/500
37/37 [==============================] - 3s 70ms/step - loss: 0.0556 - acc: 0.9875 - val_loss: 1.1948 - val_acc: 0.7154

Epoch 00052: val_loss did not improve from 0.51122
Epoch 53/500
37/37 [==============================] - 3s 69ms/step - loss: 0.0579 - acc: 0.9873 - val_loss: 1.1924 - val_acc: 0.7225

Epoch 00053: val_loss did not improve from 0.51122
Epoch 54/500
37/37 [==============================] - 2s 68ms/step - loss: 0.0545 - acc: 0.9890 - val_loss: 1.2052 - val_acc: 0.7251

Epoch 00054: val_loss did not improve from 0.51122
Epoch 55/500
37/37 [==============================] - 3s 68ms/step - loss: 0.0528 - acc: 0.9894 - val_loss: 1.2172 - val_acc: 0.7238

Epoch 00055: val_loss did not improve from 0.51122
Epoch 00055: early stopping
############# model path :  ./models/seed=6_05_val_loss:0.51

37/37 [==============================] - 3s 69ms/step - loss: 0.1107 - acc: 0.9717 - val_loss: 0.7844 - val_acc: 0.7594

Epoch 00031: val_loss did not improve from 0.49657
Epoch 32/500
37/37 [==============================] - 3s 68ms/step - loss: 0.0984 - acc: 0.9748 - val_loss: 0.8271 - val_acc: 0.7587

Epoch 00032: val_loss did not improve from 0.49657
Epoch 33/500
37/37 [==============================] - 3s 69ms/step - loss: 0.0927 - acc: 0.9771 - val_loss: 0.8574 - val_acc: 0.7561

Epoch 00033: val_loss did not improve from 0.49657
Epoch 34/500
37/37 [==============================] - 3s 69ms/step - loss: 0.0914 - acc: 0.9784 - val_loss: 0.8898 - val_acc: 0.7568

Epoch 00034: val_loss did not improve from 0.49657
Epoch 35/500
37/37 [==============================] - 3s 68ms/step - loss: 0.0862 - acc: 0.9786 - val_loss: 0.9106 - val_acc: 0.7555

Epoch 00035: val_loss did not improve from 0.49657
Epoch 36/500
37/37 [==============================] - 3s 68ms/step - loss: 0.0852 - acc:

37/37 [==============================] - 3s 68ms/step - loss: 0.2702 - acc: 0.8947 - val_loss: 0.7479 - val_acc: 0.6915

Epoch 00011: val_loss did not improve from 0.49194
Epoch 12/500
37/37 [==============================] - 3s 68ms/step - loss: 0.3053 - acc: 0.8822 - val_loss: 0.6281 - val_acc: 0.7477

Epoch 00012: val_loss did not improve from 0.49194
Epoch 13/500
37/37 [==============================] - 3s 69ms/step - loss: 0.2706 - acc: 0.8917 - val_loss: 0.6445 - val_acc: 0.7451

Epoch 00013: val_loss did not improve from 0.49194
Epoch 14/500
37/37 [==============================] - 3s 69ms/step - loss: 0.2326 - acc: 0.9154 - val_loss: 0.6207 - val_acc: 0.7536

Epoch 00014: val_loss did not improve from 0.49194
Epoch 15/500
37/37 [==============================] - 3s 69ms/step - loss: 0.1760 - acc: 0.9463 - val_loss: 0.6991 - val_acc: 0.7348

Epoch 00015: val_loss did not improve from 0.49194
Epoch 16/500
37/37 [==============================] - 3s 68ms/step - loss: 0.1592 - acc:

정확도 : 0.7652005174644243, seed_num = 8
Epoch 1/500
37/37 [==============================] - 6s 79ms/step - loss: 0.6713 - acc: 0.6153 - val_loss: 0.6805 - val_acc: 0.5899

Epoch 00001: val_loss improved from inf to 0.68054, saving model to ./models/seed=9_01_val_loss:0.6805.hdf5
Epoch 2/500
37/37 [==============================] - 2s 67ms/step - loss: 0.6612 - acc: 0.6142 - val_loss: 0.6390 - val_acc: 0.5899

Epoch 00002: val_loss improved from 0.68054 to 0.63900, saving model to ./models/seed=9_02_val_loss:0.6390.hdf5
Epoch 3/500
37/37 [==============================] - 3s 70ms/step - loss: 0.5530 - acc: 0.7165 - val_loss: 0.5425 - val_acc: 0.7354

Epoch 00003: val_loss improved from 0.63900 to 0.54247, saving model to ./models/seed=9_03_val_loss:0.5425.hdf5
Epoch 4/500
37/37 [==============================] - 3s 70ms/step - loss: 0.4426 - acc: 0.8078 - val_loss: 0.5444 - val_acc: 0.7477

Epoch 00004: val_loss did not improve from 0.54247
Epoch 5/500
37/37 [===========================

37/37 [==============================] - 3s 69ms/step - loss: 0.0643 - acc: 0.9855 - val_loss: 1.1602 - val_acc: 0.7270

Epoch 00038: val_loss did not improve from 0.54247
Epoch 39/500
37/37 [==============================] - 3s 69ms/step - loss: 0.0645 - acc: 0.9843 - val_loss: 1.1313 - val_acc: 0.7316

Epoch 00039: val_loss did not improve from 0.54247
Epoch 40/500
37/37 [==============================] - 3s 69ms/step - loss: 0.0658 - acc: 0.9840 - val_loss: 1.1214 - val_acc: 0.7290

Epoch 00040: val_loss did not improve from 0.54247
Epoch 41/500
37/37 [==============================] - 3s 69ms/step - loss: 0.0677 - acc: 0.9836 - val_loss: 1.1902 - val_acc: 0.7219

Epoch 00041: val_loss did not improve from 0.54247
Epoch 42/500
37/37 [==============================] - 3s 69ms/step - loss: 0.0688 - acc: 0.9834 - val_loss: 1.1946 - val_acc: 0.7206

Epoch 00042: val_loss did not improve from 0.54247
Epoch 43/500
37/37 [==============================] - 3s 68ms/step - loss: 0.0689 - acc:

- 정확도 df 만들고 평균 확인 (val_acc) : [0.75789133]
- 정확도 df 만들고 평균 확인 (val_loss) : [0.76073739]


In [4]:
df

,0
0,0.760673
1,0.772316
2,0.758085
3,0.727038
4,0.778137
5,0.771669
6,0.773609
7,0.756145
8,0.765201
9,0.744502


In [48]:
df

,0
0,0.758085
1,0.772962
2,0.756792
3,0.751617
4,0.774903
5,0.774256
6,0.773609
7,0.733506
8,0.760673
9,0.722510


In [8]:
# RNN model
def rnn_model():
    seed_num = 42 # model_seed
    tf.random.set_seed(seed_num)

    rnn = Sequential()
    rnn.add(InputLayer(input_shape=(X_train.shape[1],X_train.shape[2])))
    rnn.add(SimpleRNN(units=128, activation='relu', return_sequences=True))
    rnn.add(SimpleRNN(units=64, activation='relu', return_sequences=True))
    rnn.add(SimpleRNN(units=32, activation='relu', return_sequences=False))
    rnn.add(Dropout(0.2))
    rnn.add(Dense(units=1))
    return rnn

def gru_model(X_train):
    seed_num = 42 # model_seed
    tf.random.set_seed(seed_num)

    gru = Sequential()
    gru.add(InputLayer(input_shape=(X_train.shape[1],X_train.shape[2])))
    gru.add(GRU(units=128, activation='relu', return_sequences=True))
    gru.add(GRU(units=64, activation='relu', return_sequences=True))
    gru.add(GRU(units=32, activation='relu', return_sequences=False))
    gru.add(Dropout(0.2))
    gru.add(Dense(units=1))
    return gru

def sorted_ls(path):
    mtime=lambda f: os.stat(os.path.join(path, f)).st_mtime
    return list(sorted(os.listdir(path), key=mtime))

In [9]:
def time_series_cv(name):
    path = '/project/LSH/'
    with tf.device('/device:GPU:0'):
        # 1. Data load
        x = np.load(path + 'x_(7727,10,4068).npy')
        y = np.load(path + 'y_(7727,1).npy')

        # 3-1. Best model saving
        MODEL_SAVE_FOLDER_PATH = './models/'
        if not os.path.exists(MODEL_SAVE_FOLDER_PATH):
            os.mkdir(MODEL_SAVE_FOLDER_PATH)

        data={}
        sss = StratifiedShuffleSplit(n_splits=10, test_size = 0.2, random_state = 42)

        # 4. Crossvalidation
        for seed, (train_index, test_index) in tqdm(enumerate(sss.split(x, y))):
            X_train, y_train = x[train_index,:,:], y[train_index]
            X_test, y_test = x[test_index,:,:], y[test_index]

            # 3-2. Best model saving
#             model_path = MODEL_SAVE_FOLDER_PATH + name +'{epoch:02d}_val_acc:{val_acc:.4f}.hdf5'

#             cb_checkpoint = ModelCheckpoint(filepath=model_path, monitor='val_acc', verbose=1, save_best_only=True)
            early_stop = EarlyStopping(monitor='val_loss', patience=50, verbose=1, restore_best_weights=False)

            if name == 'LSTM':
                model = lstm_model()
            elif name == 'RNN':
                model = rnn_model()
            elif name == 'GRU':
                model = gru_model(X_train)
                
            model.compile(optimizer= keras.optimizers.Adam(learning_rate = 0.001), 
                          loss = "binary_crossentropy", metrics=['acc'])
            model.fit(X_train, y_train, validation_split=0.25, batch_size=128, epochs=500,
                      callbacks=[early_stop], shuffle=False)

#             5. accuracy checking 
            if name == 'LSTM':
                from pathlib import Path
                paths = sorted(Path(MODEL_SAVE_FOLDER_PATH).iterdir(), key=os.path.getmtime)[-1]
                best_model_path = str(paths)
                best_model_path = MODEL_SAVE_FOLDER_PATH + sorted_ls('models/')[-1]
                print('############# model path : ', best_model_path)

                best_model = keras.models.load_model(best_model_path) 
                y_pred_test = best_model.predict(X_test, batch_size=10000, workers=-1, use_multiprocessing=True)
            else:    
                y_pred_test = model.predict(X_test, batch_size=10000, workers=-1, use_multiprocessing=True)

            y_pred_test[y_pred_test>0.5]=1
            y_pred_test[y_pred_test<=0.5]=0

            acc = accuracy_score(y_test, y_pred_test)
            data[seed]=acc
            print(f'정확도 : {acc}, seed_num = {seed}')

        df = pd.DataFrame.from_dict(data, orient='index')
        print(f'정확도 df 만들고 평균 확인 : {df.mean().values}')    
    return df

In [10]:
rnn_df = time_series_cv('RNN')
rnn_df

0it [00:00, ?it/s]

Epoch 1/500
37/37 [==============================] - 2s 31ms/step - loss: 0.9663 - acc: 0.5946 - val_loss: 0.6164 - val_acc: 0.6442
Epoch 2/500
37/37 [==============================] - 1s 23ms/step - loss: 0.5653 - acc: 0.7273 - val_loss: 0.6040 - val_acc: 0.7186
Epoch 3/500
37/37 [==============================] - 1s 24ms/step - loss: 0.4457 - acc: 0.8291 - val_loss: 0.8773 - val_acc: 0.7160
Epoch 4/500
37/37 [==============================] - 1s 24ms/step - loss: 0.3394 - acc: 0.8844 - val_loss: 1.1433 - val_acc: 0.7245
Epoch 5/500
37/37 [==============================] - 1s 24ms/step - loss: 0.3280 - acc: 0.9027 - val_loss: 1.6274 - val_acc: 0.6992
Epoch 6/500
37/37 [==============================] - 1s 24ms/step - loss: 0.3259 - acc: 0.8820 - val_loss: 1.1337 - val_acc: 0.7238
Epoch 7/500
37/37 [==============================] - 1s 24ms/step - loss: 0.2417 - acc: 0.9318 - val_loss: 1.2688 - val_acc: 0.7193
Epoch 8/500
37/37 [==============================] - 1s 21ms/step - loss: 0.

37/37 [==============================] - 1s 24ms/step - loss: 0.0970 - acc: 0.9696 - val_loss: 2.1266 - val_acc: 0.7309
Epoch 11/500
37/37 [==============================] - 1s 24ms/step - loss: 0.0855 - acc: 0.9761 - val_loss: 2.3544 - val_acc: 0.7257
Epoch 12/500
37/37 [==============================] - 1s 22ms/step - loss: 0.0511 - acc: 0.9847 - val_loss: 2.5556 - val_acc: 0.7367
Epoch 13/500
37/37 [==============================] - 1s 22ms/step - loss: 0.0439 - acc: 0.9873 - val_loss: 2.7165 - val_acc: 0.7342
Epoch 14/500
37/37 [==============================] - 1s 23ms/step - loss: 0.0424 - acc: 0.9894 - val_loss: 2.8356 - val_acc: 0.7251
Epoch 15/500
37/37 [==============================] - 1s 24ms/step - loss: 0.0373 - acc: 0.9922 - val_loss: 2.8589 - val_acc: 0.7303
Epoch 16/500
37/37 [==============================] - 1s 24ms/step - loss: 0.0318 - acc: 0.9948 - val_loss: 3.0395 - val_acc: 0.7290
Epoch 17/500
37/37 [==============================] - 1s 23ms/step - loss: 0.0323 

37/37 [==============================] - 1s 23ms/step - loss: 0.0402 - acc: 0.9950 - val_loss: 2.4732 - val_acc: 0.7406
Epoch 19/500
37/37 [==============================] - 1s 23ms/step - loss: 0.0374 - acc: 0.9968 - val_loss: 2.5778 - val_acc: 0.7426
Epoch 20/500
37/37 [==============================] - 1s 20ms/step - loss: 0.0352 - acc: 0.9972 - val_loss: 2.6316 - val_acc: 0.7419
Epoch 21/500
37/37 [==============================] - 1s 22ms/step - loss: 0.0344 - acc: 0.9970 - val_loss: 2.6207 - val_acc: 0.7451
Epoch 22/500
37/37 [==============================] - 1s 24ms/step - loss: 0.0335 - acc: 0.9981 - val_loss: 2.7049 - val_acc: 0.7439
Epoch 23/500
37/37 [==============================] - 1s 23ms/step - loss: 0.0324 - acc: 0.9978 - val_loss: 2.8008 - val_acc: 0.7419
Epoch 24/500
37/37 [==============================] - 1s 23ms/step - loss: 0.0321 - acc: 0.9981 - val_loss: 2.7796 - val_acc: 0.7439
Epoch 25/500
37/37 [==============================] - 1s 24ms/step - loss: 0.0316 

37/37 [==============================] - 1s 20ms/step - loss: 0.0096 - acc: 0.9991 - val_loss: 2.8913 - val_acc: 0.7464
Epoch 29/500
37/37 [==============================] - 1s 21ms/step - loss: 0.0097 - acc: 0.9996 - val_loss: 2.9415 - val_acc: 0.7380
Epoch 30/500
37/37 [==============================] - 1s 23ms/step - loss: 0.0086 - acc: 0.9994 - val_loss: 2.9534 - val_acc: 0.7451
Epoch 31/500
37/37 [==============================] - 1s 23ms/step - loss: 0.0083 - acc: 0.9994 - val_loss: 2.9166 - val_acc: 0.7406
Epoch 32/500
37/37 [==============================] - 1s 23ms/step - loss: 0.0083 - acc: 0.9991 - val_loss: 2.9076 - val_acc: 0.7374
Epoch 33/500
37/37 [==============================] - 1s 19ms/step - loss: 0.0081 - acc: 0.9994 - val_loss: 2.9103 - val_acc: 0.7419
Epoch 34/500
37/37 [==============================] - 1s 22ms/step - loss: 0.0078 - acc: 0.9996 - val_loss: 3.0461 - val_acc: 0.7413
Epoch 35/500
37/37 [==============================] - 1s 23ms/step - loss: 0.0080 

...

37/37 [==============================] - 1s 25ms/step - loss: 0.0212 - acc: 0.9987 - val_loss: 3.0202 - val_acc: 0.7167
Epoch 47/500
37/37 [==============================] - 1s 24ms/step - loss: 0.0188 - acc: 0.9987 - val_loss: 3.0771 - val_acc: 0.7147
Epoch 48/500
37/37 [==============================] - 1s 24ms/step - loss: 0.0182 - acc: 0.9987 - val_loss: 3.1215 - val_acc: 0.7186
Epoch 49/500
37/37 [==============================] - 1s 23ms/step - loss: 0.0209 - acc: 0.9985 - val_loss: 3.1384 - val_acc: 0.7180
Epoch 50/500
37/37 [==============================] - 1s 22ms/step - loss: 0.0213 - acc: 0.9985 - val_loss: 3.1316 - val_acc: 0.7160
Epoch 51/500
37/37 [==============================] - 1s 22ms/step - loss: 0.0182 - acc: 0.9985 - val_loss: 3.1459 - val_acc: 0.7180
Epoch 52/500
37/37 [==============================] - 1s 23ms/step - loss: 0.0175 - acc: 0.9989 - val_loss: 3.1710 - val_acc: 0.7180
Epoch 00052: early stopping
정확도 : 0.7322121604139715, seed_num = 5
Epoch 1/500
37/

Epoch 3/500
37/37 [==============================] - 1s 22ms/step - loss: 0.6274 - acc: 0.6716 - val_loss: 0.5616 - val_acc: 0.7180
Epoch 4/500
37/37 [==============================] - 1s 23ms/step - loss: 0.5058 - acc: 0.7745 - val_loss: 0.5742 - val_acc: 0.7361
Epoch 5/500
37/37 [==============================] - 1s 24ms/step - loss: 0.3957 - acc: 0.8410 - val_loss: 1.2367 - val_acc: 0.7400
Epoch 6/500
37/37 [==============================] - 1s 24ms/step - loss: 0.3085 - acc: 0.8915 - val_loss: 1.4532 - val_acc: 0.7587
Epoch 7/500
37/37 [==============================] - 1s 22ms/step - loss: 0.3613 - acc: 0.8630 - val_loss: 0.7773 - val_acc: 0.7497
Epoch 8/500
37/37 [==============================] - 1s 23ms/step - loss: 0.2776 - acc: 0.9090 - val_loss: 1.0796 - val_acc: 0.7529
Epoch 9/500
37/37 [==============================] - 1s 23ms/step - loss: 0.1714 - acc: 0.9538 - val_loss: 1.5208 - val_acc: 0.7633
Epoch 10/500
37/37 [==============================] - 1s 23ms/step - loss: 0

37/37 [==============================] - 1s 22ms/step - loss: 0.0868 - acc: 0.9823 - val_loss: 2.3140 - val_acc: 0.7025
Epoch 12/500
37/37 [==============================] - 1s 23ms/step - loss: 0.0743 - acc: 0.9875 - val_loss: 2.8269 - val_acc: 0.7173
Epoch 13/500
37/37 [==============================] - 1s 22ms/step - loss: 0.0458 - acc: 0.9918 - val_loss: 2.7714 - val_acc: 0.7206
Epoch 14/500
37/37 [==============================] - 1s 23ms/step - loss: 0.0364 - acc: 0.9961 - val_loss: 2.9601 - val_acc: 0.7186
Epoch 15/500
37/37 [==============================] - 1s 23ms/step - loss: 0.0341 - acc: 0.9972 - val_loss: 3.0212 - val_acc: 0.7180
Epoch 16/500
37/37 [==============================] - 1s 23ms/step - loss: 0.0328 - acc: 0.9974 - val_loss: 3.0806 - val_acc: 0.7206
Epoch 17/500
37/37 [==============================] - 1s 21ms/step - loss: 0.0319 - acc: 0.9974 - val_loss: 3.1561 - val_acc: 0.7160
Epoch 18/500
37/37 [==============================] - 1s 23ms/step - loss: 0.0308 

37/37 [==============================] - 1s 22ms/step - loss: 0.0354 - acc: 0.9976 - val_loss: 2.7451 - val_acc: 0.7439
Epoch 22/500
37/37 [==============================] - 1s 22ms/step - loss: 0.0349 - acc: 0.9972 - val_loss: 2.7738 - val_acc: 0.7471
Epoch 23/500
37/37 [==============================] - 1s 22ms/step - loss: 0.0345 - acc: 0.9974 - val_loss: 2.8134 - val_acc: 0.7432
Epoch 24/500
37/37 [==============================] - 1s 24ms/step - loss: 0.0338 - acc: 0.9978 - val_loss: 2.8544 - val_acc: 0.7426
Epoch 25/500
37/37 [==============================] - 1s 23ms/step - loss: 0.0344 - acc: 0.9978 - val_loss: 2.8129 - val_acc: 0.7432
Epoch 26/500
37/37 [==============================] - 1s 24ms/step - loss: 0.0342 - acc: 0.9978 - val_loss: 2.7854 - val_acc: 0.7426
Epoch 27/500
37/37 [==============================] - 1s 22ms/step - loss: 0.0344 - acc: 0.9976 - val_loss: 2.8512 - val_acc: 0.7458
Epoch 28/500
37/37 [==============================] - 1s 23ms/step - loss: 0.0340 

,0
0,0.730272
1,0.737387
2,0.717335
3,0.734153
4,0.731565
5,0.732212
6,0.724450
7,0.736740
8,0.747736
9,0.723157


In [11]:
gru_df = time_series_cv('GRU')
gru_df

0it [00:00, ?it/s]

Epoch 1/500
37/37 [==============================] - 4s 54ms/step - loss: 1.0184 - acc: 0.5618 - val_loss: 0.6212 - val_acc: 0.6527
Epoch 2/500
37/37 [==============================] - 2s 45ms/step - loss: 0.5801 - acc: 0.6867 - val_loss: 0.5735 - val_acc: 0.6953
Epoch 3/500
37/37 [==============================] - 2s 47ms/step - loss: 0.4919 - acc: 0.7735 - val_loss: 0.7082 - val_acc: 0.7296
Epoch 4/500
37/37 [==============================] - 2s 47ms/step - loss: 0.3928 - acc: 0.8324 - val_loss: 0.8983 - val_acc: 0.7135
Epoch 5/500
37/37 [==============================] - 2s 47ms/step - loss: 0.4918 - acc: 0.7560 - val_loss: 1.0973 - val_acc: 0.6740
Epoch 6/500
37/37 [==============================] - 2s 46ms/step - loss: 0.4283 - acc: 0.8076 - val_loss: 0.9734 - val_acc: 0.7199
Epoch 7/500
37/37 [==============================] - 2s 47ms/step - loss: 0.3197 - acc: 0.8712 - val_loss: 1.4437 - val_acc: 0.7212
Epoch 8/500
37/37 [==============================] - 2s 47ms/step - loss: 0.

Epoch 3/500
37/37 [==============================] - 2s 46ms/step - loss: 0.5190 - acc: 0.7655 - val_loss: 0.5528 - val_acc: 0.7277
Epoch 4/500
37/37 [==============================] - 2s 46ms/step - loss: 0.4617 - acc: 0.8088 - val_loss: 0.7187 - val_acc: 0.7658
Epoch 5/500
37/37 [==============================] - 2s 48ms/step - loss: 0.4018 - acc: 0.8606 - val_loss: 0.8727 - val_acc: 0.7497
Epoch 6/500
37/37 [==============================] - 2s 46ms/step - loss: 0.3515 - acc: 0.8887 - val_loss: 1.1937 - val_acc: 0.7484
Epoch 7/500
37/37 [==============================] - 2s 46ms/step - loss: 0.2764 - acc: 0.9266 - val_loss: 2.4761 - val_acc: 0.7193
Epoch 8/500
37/37 [==============================] - 2s 45ms/step - loss: 0.2202 - acc: 0.9525 - val_loss: 2.2573 - val_acc: 0.7516
Epoch 9/500
37/37 [==============================] - 2s 48ms/step - loss: 0.2004 - acc: 0.9493 - val_loss: 2.1430 - val_acc: 0.7581
Epoch 10/500
37/37 [==============================] - 2s 45ms/step - loss: 0

37/37 [==============================] - 2s 46ms/step - loss: 0.2936 - acc: 0.8876 - val_loss: 1.3135 - val_acc: 0.7516
Epoch 8/500
37/37 [==============================] - 2s 48ms/step - loss: 0.2037 - acc: 0.9249 - val_loss: 1.7783 - val_acc: 0.7542
Epoch 9/500
37/37 [==============================] - 2s 48ms/step - loss: 0.1426 - acc: 0.9553 - val_loss: 2.1840 - val_acc: 0.7477
Epoch 10/500
37/37 [==============================] - 2s 48ms/step - loss: 0.1239 - acc: 0.9698 - val_loss: 2.2260 - val_acc: 0.7296
Epoch 11/500
37/37 [==============================] - 2s 48ms/step - loss: 0.1015 - acc: 0.9752 - val_loss: 2.6451 - val_acc: 0.7451
Epoch 12/500
37/37 [==============================] - 2s 48ms/step - loss: 0.1051 - acc: 0.9763 - val_loss: 2.4723 - val_acc: 0.7387
Epoch 13/500
37/37 [==============================] - 2s 48ms/step - loss: 0.0825 - acc: 0.9795 - val_loss: 2.8259 - val_acc: 0.7393
Epoch 14/500
37/37 [==============================] - 2s 48ms/step - loss: 0.1026 - 

37/37 [==============================] - 2s 44ms/step - loss: 0.1828 - acc: 0.9320 - val_loss: 1.7593 - val_acc: 0.7180
Epoch 10/500
37/37 [==============================] - 2s 44ms/step - loss: 0.1533 - acc: 0.9517 - val_loss: 1.6710 - val_acc: 0.7232
Epoch 11/500
37/37 [==============================] - 2s 46ms/step - loss: 0.1298 - acc: 0.9590 - val_loss: 2.0369 - val_acc: 0.7529
Epoch 12/500
37/37 [==============================] - 2s 45ms/step - loss: 0.0913 - acc: 0.9773 - val_loss: 2.4523 - val_acc: 0.7432
Epoch 13/500
37/37 [==============================] - 2s 46ms/step - loss: 0.1616 - acc: 0.9609 - val_loss: 1.9536 - val_acc: 0.7432
Epoch 14/500
37/37 [==============================] - 2s 46ms/step - loss: 0.0854 - acc: 0.9754 - val_loss: 2.2902 - val_acc: 0.7471
Epoch 15/500
37/37 [==============================] - 2s 45ms/step - loss: 0.0634 - acc: 0.9825 - val_loss: 2.4867 - val_acc: 0.7419
Epoch 16/500
37/37 [==============================] - 2s 45ms/step - loss: 0.0551 

...

37/37 [==============================] - 2s 47ms/step - loss: 0.0703 - acc: 0.9896 - val_loss: 3.1358 - val_acc: 0.7393
Epoch 22/500
37/37 [==============================] - 2s 47ms/step - loss: 0.0728 - acc: 0.9896 - val_loss: 3.0836 - val_acc: 0.7523
Epoch 23/500
37/37 [==============================] - 2s 49ms/step - loss: 0.0668 - acc: 0.9912 - val_loss: 3.1069 - val_acc: 0.7426
Epoch 24/500
37/37 [==============================] - 2s 45ms/step - loss: 0.0695 - acc: 0.9922 - val_loss: 3.0398 - val_acc: 0.7523
Epoch 25/500
37/37 [==============================] - 2s 47ms/step - loss: 0.0903 - acc: 0.9853 - val_loss: 2.8190 - val_acc: 0.7497
Epoch 26/500
37/37 [==============================] - 2s 46ms/step - loss: 0.0792 - acc: 0.9888 - val_loss: 2.7950 - val_acc: 0.7542
Epoch 27/500
37/37 [==============================] - 2s 48ms/step - loss: 0.0940 - acc: 0.9877 - val_loss: 2.9892 - val_acc: 0.7419
Epoch 28/500
37/37 [==============================] - 2s 45ms/step - loss: 0.0998 

37/37 [==============================] - 2s 46ms/step - loss: 0.0870 - acc: 0.9855 - val_loss: 2.8630 - val_acc: 0.7484
Epoch 28/500
37/37 [==============================] - 2s 45ms/step - loss: 0.0792 - acc: 0.9860 - val_loss: 2.8538 - val_acc: 0.7484
Epoch 29/500
37/37 [==============================] - 2s 46ms/step - loss: 0.0789 - acc: 0.9853 - val_loss: 2.8230 - val_acc: 0.7393
Epoch 30/500
37/37 [==============================] - 2s 48ms/step - loss: 0.0797 - acc: 0.9855 - val_loss: 2.8972 - val_acc: 0.7458
Epoch 31/500
37/37 [==============================] - 2s 46ms/step - loss: 0.0834 - acc: 0.9860 - val_loss: 3.0768 - val_acc: 0.7490
Epoch 32/500
37/37 [==============================] - 2s 46ms/step - loss: 0.0823 - acc: 0.9881 - val_loss: 3.1277 - val_acc: 0.7458
Epoch 33/500
37/37 [==============================] - 2s 45ms/step - loss: 0.0805 - acc: 0.9890 - val_loss: 3.2425 - val_acc: 0.7439
Epoch 34/500
37/37 [==============================] - 2s 47ms/step - loss: 0.0793 

37/37 [==============================] - 2s 47ms/step - loss: 0.0653 - acc: 0.9903 - val_loss: 3.0195 - val_acc: 0.7503
Epoch 33/500
37/37 [==============================] - 2s 47ms/step - loss: 0.0627 - acc: 0.9920 - val_loss: 3.1740 - val_acc: 0.7477
Epoch 34/500
37/37 [==============================] - 2s 45ms/step - loss: 0.0605 - acc: 0.9933 - val_loss: 3.2173 - val_acc: 0.7490
Epoch 35/500
37/37 [==============================] - 2s 46ms/step - loss: 0.0599 - acc: 0.9933 - val_loss: 3.2683 - val_acc: 0.7490
Epoch 36/500
37/37 [==============================] - 2s 47ms/step - loss: 0.0597 - acc: 0.9937 - val_loss: 3.3454 - val_acc: 0.7471
Epoch 37/500
37/37 [==============================] - 2s 46ms/step - loss: 0.0588 - acc: 0.9942 - val_loss: 3.4157 - val_acc: 0.7439
Epoch 38/500
37/37 [==============================] - 2s 48ms/step - loss: 0.0588 - acc: 0.9944 - val_loss: 3.3855 - val_acc: 0.7458
Epoch 39/500
37/37 [==============================] - 2s 45ms/step - loss: 0.0549 

37/37 [==============================] - 2s 46ms/step - loss: 0.0306 - acc: 0.9965 - val_loss: 3.2798 - val_acc: 0.7354
Epoch 37/500
37/37 [==============================] - 2s 46ms/step - loss: 0.0334 - acc: 0.9961 - val_loss: 3.2720 - val_acc: 0.7419
Epoch 38/500
37/37 [==============================] - 2s 45ms/step - loss: 0.0250 - acc: 0.9981 - val_loss: 3.3191 - val_acc: 0.7426
Epoch 39/500
37/37 [==============================] - 2s 46ms/step - loss: 1.8272 - acc: 0.8595 - val_loss: 3.9469 - val_acc: 0.6940
Epoch 40/500
37/37 [==============================] - 2s 46ms/step - loss: 1.1479 - acc: 0.8576 - val_loss: 1.5524 - val_acc: 0.7277
Epoch 41/500
37/37 [==============================] - 2s 46ms/step - loss: 0.3923 - acc: 0.9146 - val_loss: 1.6460 - val_acc: 0.7419
Epoch 42/500
37/37 [==============================] - 2s 47ms/step - loss: 0.2051 - acc: 0.9577 - val_loss: 2.3999 - val_acc: 0.7529
Epoch 43/500
37/37 [==============================] - 2s 46ms/step - loss: 0.1462 

,0
0,0.744502
1,0.741915
2,0.746442
3,0.750323
4,0.743855
5,0.748383
6,0.746442
7,0.749677
8,0.747736
9,0.729625


# 비시계열

In [29]:
def non_time_series_cv(model):
    path = '/project/LSH/'
    with tf.device('/device:GPU:0'):
        # 1. Data load
        x = np.load(path + 'x_(7727,4068).npy')
        y = np.load(path + 'y_(7727,1).npy')

        # 3-1. Best model saving
        MODEL_SAVE_FOLDER_PATH = './models/'
        if not os.path.exists(MODEL_SAVE_FOLDER_PATH):
            os.mkdir(MODEL_SAVE_FOLDER_PATH)

        acc_list = []
        sss = StratifiedShuffleSplit(n_splits=10, test_size = 0.2, random_state = 42)

        # 4. Crossvalidation
        for seed, (train_index, test_index) in tqdm(enumerate(sss.split(x, y))):
            X_train, y_train = x[train_index,:], y[train_index]
            X_test, y_test = x[test_index,:], y[test_index]
            model.fit(X_train, y_train)
            pred = model.predict(X_test)
            acc = accuracy_score(y_test, pred)
            print(f'{str(model)} : {acc}')
            acc_list.append(acc)
        mean_pred = np.mean(acc_list)
        print(f'{str(model)} MEAN : {mean_pred}')
        return mean_pred

In [30]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

result = []

# Support vector machine
model = SVC()
result.append({'model' : str(model), 'mean' : non_time_series_cv(model)})

# Gaussian naive bayes
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
result.append({'model' : str(model), 'mean' : non_time_series_cv(model)})

# K nearest neighbor
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier()
result.append({'model' : str(model), 'mean' : non_time_series_cv(model)})

# Random Forest
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
result.append({'model' : str(model), 'mean' : non_time_series_cv(model)})

# Gradient boosting
from sklearn.ensemble import GradientBoostingClassifier
model = GradientBoostingClassifier()
result.append({'model' : str(model), 'mean' : non_time_series_cv(model)})

# Neural network
from sklearn.neural_network import MLPClassifier
model = MLPClassifier(max_iter=1000)
result.append({'model' : str(model), 'mean' : non_time_series_cv(model)})

# AdaBoost
from sklearn.ensemble import AdaBoostClassifier
model = AdaBoostClassifier(n_estimators=500, 
                        random_state=10, 
                        learning_rate=0.1)
result.append({'model' : str(model), 'mean' : non_time_series_cv(model)})

0it [00:00, ?it/s]

SVC() : 0.779430789133247
SVC() : 0.7807244501940491
SVC() : 0.7619663648124192
SVC() : 0.7729624838292367
SVC() : 0.7613195342820182
SVC() : 0.7723156532988357
SVC() : 0.7736093143596378
SVC() : 0.7639068564036222
SVC() : 0.7749029754204398
SVC() : 0.7729624838292367
SVC() MEAN : 0.7714100905562742


0it [00:00, ?it/s]

GaussianNB() : 0.48641655886157825
GaussianNB() : 0.4689521345407503
GaussianNB() : 0.4987063389391979
GaussianNB() : 0.47347994825355755
GaussianNB() : 0.48124191461837
GaussianNB() : 0.4922380336351876
GaussianNB() : 0.49029754204398446
GaussianNB() : 0.47477360931435963
GaussianNB() : 0.4831824062095731
GaussianNB() : 0.4870633893919793
GaussianNB() MEAN : 0.48363518758085383


0it [00:00, ?it/s]

...

GradientBoostingClassifier() : 0.7561448900388098
GradientBoostingClassifier() : 0.7496765847347995
GradientBoostingClassifier() : 0.7380336351875808
GradientBoostingClassifier() : 0.740620957309185
GradientBoostingClassifier() : 0.7445019404915912
GradientBoostingClassifier() : 0.7496765847347995
GradientBoostingClassifier() : 0.7613195342820182
GradientBoostingClassifier() : 0.741267787839586
GradientBoostingClassifier() : 0.7432082794307892
GradientBoostingClassifier() : 0.7600258732212161
GradientBoostingClassifier() MEAN : 0.7484476067270374


0it [00:00, ?it/s]

MLPClassifier(max_iter=1000) : 0.7289780077619664
MLPClassifier(max_iter=1000) : 0.7438551099611902
MLPClassifier(max_iter=1000) : 0.740620957309185
MLPClassifier(max_iter=1000) : 0.7445019404915912
MLPClassifier(max_iter=1000) : 0.7153945666235446
MLPClassifier(max_iter=1000) : 0.7212160413971539
MLPClassifier(max_iter=1000) : 0.7373868046571799
MLPClassifier(max_iter=1000) : 0.7302716688227684
MLPClassifier(max_iter=1000) : 0.7289780077619664
MLPClassifier(max_iter=1000) : 0.7335058214747736
MLPClassifier(max_iter=1000) MEAN : 0.732470892626132


0it [00:00, ?it/s]

AdaBoostClassifier(learning_rate=0.1, n_estimators=500, random_state=10) : 0.7548512289780077
AdaBoostClassifier(learning_rate=0.1, n_estimators=500, random_state=10) : 0.7542043984476067
AdaBoostClassifier(learning_rate=0.1, n_estimators=500, random_state=10) : 0.7457956015523933
AdaBoostClassifier(learning_rate=0.1, n_estimators=500, random_state=10) : 0.7529107373868047
AdaBoostClassifier(learning_rate=0.1, n_estimators=500, random_state=10) : 0.7483829236739974
AdaBoostClassifier(learning_rate=0.1, n_estimators=500, random_state=10) : 0.758732212160414
AdaBoostClassifier(learning_rate=0.1, n_estimators=500, random_state=10) : 0.7690815006468306
AdaBoostClassifier(learning_rate=0.1, n_estimators=500, random_state=10) : 0.7399741267787839
AdaBoostClassifier(learning_rate=0.1, n_estimators=500, random_state=10) : 0.7399741267787839
AdaBoostClassifier(learning_rate=0.1, n_estimators=500, random_state=10) : 0.7580853816300129
AdaBoostClassifier(learning_rate=0.1, n_estimators=500, rando

In [35]:
result.append({'model' : 'LSTM', 'mean' : 0.75789133})
df = pd.DataFrame(result)
df

,model,mean
0,SVC(),0.771410
1,GaussianNB(),0.483635
2,KNeighborsClassifier(),0.659056
3,RandomForestClassifier(),0.738422
4,GradientBoostingClassifier(),0.748448
5,MLPClassifier(max_iter=1000),0.732471
6,"AdaBoostClassifier(learning_rate=0.1, n_estima...",0.752199
7,LSTM,0.757891


In [36]:
df.sort_values('mean')

,model,mean
1,GaussianNB(),0.483635
2,KNeighborsClassifier(),0.659056
5,MLPClassifier(max_iter=1000),0.732471
3,RandomForestClassifier(),0.738422
4,GradientBoostingClassifier(),0.748448
6,"AdaBoostClassifier(learning_rate=0.1, n_estima...",0.752199
7,LSTM,0.757891
0,SVC(),0.771410
